In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
from data_preprocessing import load_data, handle_outliers, preprocess_data
from model_building import build_model

In [3]:
def train_model(model, X_numerical, X_marca, X_modelo, X_versao, y, validation_split=0.2, epochs=100, batch_size=32):
    # Compile the model for regression
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Prepare categorical features for training
    X_marca = X_marca.reshape(-1, 1)
    X_modelo = X_modelo.reshape(-1, 1)
    X_versao = X_versao.reshape(-1, 1)

    # Train the model
    history = model.fit(
        [X_numerical, X_marca, X_modelo, X_versao],
        y,
        validation_split=validation_split,
        epochs=epochs,
        batch_size=batch_size
    )
    
    return history

In [4]:
# Load data
file_path = 'https://github.com/pedrogasparotti/challenges/raw/main/cars_train.csv'
df = load_data(file_path)

# Handle outliers
df = handle_outliers(df)

# Preprocess data
X_numerical, X_marca, X_modelo, X_versao, y = preprocess_data(df)

# Split the data
X_numerical_train, X_numerical_test, X_marca_train, X_marca_test, X_modelo_train, X_modelo_test, X_versao_train, X_versao_test, y_train, y_test = train_test_split(
    X_numerical, X_marca, X_modelo, X_versao, y, test_size=0.2, random_state=42
)

# Number of unique categories in each categorical feature
num_unique_categories = {
    'marca': df['marca'].nunique(),
    'modelo': df['modelo'].nunique(),
    'versao': df['versao'].nunique()
}

# Input shapes for numerical and categorical features
input_shape_numerical = (X_numerical.shape[1],)
input_shape_categorical = (1,)

# Build the model
embedding_size = 8
model = build_model(input_shape_numerical, input_shape_categorical, num_unique_categories, embedding_size)

# Train the model on the training data
history = train_model(model, X_numerical_train, X_marca_train, X_modelo_train, X_versao_train, y_train)

# Evaluate the model on the test data
mse = model.evaluate([X_numerical_test, X_marca_test, X_modelo_test, X_versao_test], y_test)
print("Mean Squared Error on Test Data:", mse)

Epoch 1/100
583/583 [==============================] - 3s 3ms/step - loss: 11838726144.0000 - val_loss: 4849203712.0000
Epoch 2/100
583/583 [==============================] - 1s 2ms/step - loss: 4446064128.0000 - val_loss: 3940249344.0000
Epoch 3/100
583/583 [==============================] - 1s 2ms/step - loss: 4059596800.0000 - val_loss: 3774769152.0000
Epoch 4/100
583/583 [==============================] - 1s 2ms/step - loss: 3865948928.0000 - val_loss: 3553519872.0000
Epoch 5/100
583/583 [==============================] - 1s 2ms/step - loss: 3601290752.0000 - val_loss: 3248888832.0000
Epoch 6/100
583/583 [==============================] - 1s 2ms/step - loss: 3216015360.0000 - val_loss: 2827715840.0000
Epoch 7/100
583/583 [==============================] - 2s 3ms/step - loss: 2720779264.0000 - val_loss: 2352852480.0000
Epoch 8/100
583/583 [==============================] - 1s 2ms/step - loss: 2248297984.0000 - val_loss: 1988789760.0000
Epoch 9/100
583/583 [==========================

583/583 [==============================] - 2s 3ms/step - loss: 1011393280.0000 - val_loss: 1125429888.0000
Epoch 70/100
583/583 [==============================] - 1s 2ms/step - loss: 1010345344.0000 - val_loss: 1129999104.0000
Epoch 71/100
583/583 [==============================] - 1s 3ms/step - loss: 1009908672.0000 - val_loss: 1130538240.0000
Epoch 72/100
583/583 [==============================] - 1s 2ms/step - loss: 1011258432.0000 - val_loss: 1128958464.0000
Epoch 73/100
583/583 [==============================] - 1s 2ms/step - loss: 1007828480.0000 - val_loss: 1134995456.0000
Epoch 74/100
583/583 [==============================] - 1s 2ms/step - loss: 1007804288.0000 - val_loss: 1129884032.0000
Epoch 75/100
583/583 [==============================] - 1s 2ms/step - loss: 1006199616.0000 - val_loss: 1128535936.0000
Epoch 76/100
583/583 [==============================] - 2s 3ms/step - loss: 1006685568.0000 - val_loss: 1144859776.0000
Epoch 77/100
583/583 [==============================]

In [5]:
# Make predictions using the trained model and see the differences
y_pred_test = model.predict([X_numerical_test, X_marca_test, X_modelo_test, X_versao_test]).flatten()
results_df = pd.DataFrame({'Actual Price': y_test, 'Predicted Price': y_pred_test})
results_df['Difference'] = results_df['Actual Price'] - results_df['Predicted Price']

print(results_df)

183/183 [==============================] - 0s 1ms/step
       Actual Price  Predicted Price    Difference
6345    41872.19512     47683.625000  -5811.429880
10578   88001.01927     88305.242188   -304.222917
27972   58321.94495     26204.031250  32117.913700
11384  114573.90920    109806.617188   4767.292012
12272  112246.43210    128804.125000 -16557.692900
...             ...              ...           ...
1434    73490.78126     72642.562500    848.218760
7663   272981.61430    244621.500000  28360.114300
14767  153616.64780    199833.343750 -46216.695950
27658   35530.24270     64061.539062 -28531.296362
27479   83775.38292     80165.609375   3609.773545

[5828 rows x 3 columns]


In [6]:
# Load test data
test_data_path = 'https://github.com/pedrogasparotti/challenges/raw/main/cars_test.csv'
df_test = load_data(test_data_path)

# Preprocess test data
X_numerical_test, X_marca_test, X_modelo_test, X_versao_test = preprocess_data(df_test)

# Make predictions on the test data
predictions = model.predict([X_numerical_test, X_marca_test, X_modelo_test, X_versao_test])

# Add predictions to a new DataFrame
df_predictions = pd.DataFrame(predictions, columns=['preco'])

# Concatenate test data with predictions DataFrame
df_result = pd.concat([df_test, df_predictions], axis=1)

df_result = df_result[['id', 'preco']]

# Save the combined DataFrame to a new CSV file
predictions_file_path = 'predicted.csv'
df_result.to_csv(predictions_file_path, index=False)

print("Predictions saved to:", predictions_file_path)

309/309 [==============================] - 1s 2ms/step
Predictions saved to: predicted.csv
